# Setting the Influent in Sumo - Introductory Practice and Example

In [ ]:
from dynamita.sumo import *

import numpy
import time
import matplotlib.pyplot as plt
%matplotlib notebook

In [ ]:
sumo = Sumo(sumoPath="C:/Users/Sara/AppData/Local/Dynamita/Sumo19",
           licenseFile=r"C:/Users/Sara/Desktop/sewerWRRF/networklicense.sumolic")

In [ ]:
def datacomm_callback(sumo):
    t.append(sumo.core.csumo_var_get_time_double(sumo.handle))
    snhx_effl.append(sumo.core.csumo_var_get_pvtarray_pos(sumo.handle, snhx_effl_pos, 0))
    xtss_effl.append(sumo.core.csumo_var_get_pvtarray_pos(sumo.handle, xtss_effl_pos, 0))
    return 0